## Use Polyjuice to create CADs

In [ ]:
import pandas as pd

In [ ]:
CONSTRUCTS = ['sexism', 'hate speech']

In [ ]:
training_data_path = {}
data = {}

In [ ]:
for construct in CONSTRUCTS:
    training_data_path[construct] = DATAROOT + construct + '/train/original.csv'
    data[construct] = pd.read_csv(training_data_path[construct], sep = '\t')

In [ ]:
data['sexism'].head()

In [ ]:
texts = {}

for construct in CONSTRUCTS:
    texts[construct] = dict(zip(data[construct]['_id'], data[construct]['text']))

In [ ]:
def generate_polyjuice_cad(texts):
    from polyjuice import Polyjuice
    pj = Polyjuice(model_path="uw-hai/polyjuice", is_cuda=False)

    codes = ['resemantic', 'restructure', 'negation', 'insert', 'lexical', 'shuffle', 'quantifier', 'delete']
    
    all_perturbations = {}

    for id_, text in texts.items():
        all_perturbations[id_] = {}
        for code in codes:
            all_perturbations[id_][code] = []

    for n, (id_, text) in enumerate(texts.items()):
        # print(text)
        if n % 100 == 0:
            print(n, 'done...')
        for n in range(5):  
            for code in codes:
                try:
                    all_perturbations[id_][code].extend(pj.perturb(orig_sent=text,
                                                                ctrl_code=code
                                                               ))
                except Exception as e:
                    print(text)
                    print(e)
                    pass
                
    return all_perturbations

In [ ]:
cad = {}

In [ ]:
import time
import pickle

start_time = time.time()
for construct in CONSTRUCTS:
    cad[construct] = generate_polyjuice_cad(texts[construct])
    print("--- %s seconds ---" % (time.time() - start_time))
    with open(DESIGNEDDATAROOT + '/cad/%s/' %construct + 'polyjuice.pickle', 'wb') as handle:
        pickle.dump(cad[construct], handle, protocol=pickle.HIGHEST_PROTOCOL)